In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
import yfinance as yf

In [83]:
S0 = 3110.87              # spot stock price
K = 3100.00                # strike ATM
T = 4/52                 # maturity 
r = 0.0174                 # risk free rate 
sig = 0.3575               # diffusion coefficient or volatility
N = 3                   # number of periods or number of time steps  
payoff = 'put'          # payoff

In [86]:
def option_bs(S, K, T, r, sig, payoff):
    
    d1 = (np.log(S / K) + (r + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    if payoff == "call":
        option_value = np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        option_value = np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return option_value

In [87]:
option_bs(3110.87, 3100, 4/52, 0.0174, 0.3575, 'put')

0.4996399908424095

In [65]:
def mcs_simulation_np(p):
    M = p
    I = p
    dt = T / M 
    S = np.zeros((M + 1, I))
    S[0] = S0 
    rn = np.random.standard_normal(S.shape) 
    for t in range(1, M + 1): 
        S[t] = S[t-1] * np.exp((r - sigma ** 2 / 2) * dt + sigma * np.sqrt(dt) * rn[t]) 
    return S

In [66]:
T = 4/52
r = 0.0174
sigma = 0.3575
S0 = 3110.87
K = 3100.00

In [67]:
S = mcs_simulation_np(10000)

In [68]:
S = np.transpose(S)
S

array([[3110.87      , 3108.69679263, 3109.15440629, ..., 3050.05318053,
        3045.88831491, 3049.5447373 ],
       [3110.87      , 3110.19303764, 3111.86044883, ..., 3437.01454846,
        3436.50214867, 3436.84094919],
       [3110.87      , 3113.22010027, 3115.08043383, ..., 2676.56599663,
        2676.21403311, 2676.56515634],
       ...,
       [3110.87      , 3108.90373539, 3111.1811832 , ..., 2661.92593626,
        2663.83650469, 2663.42484979],
       [3110.87      , 3111.58020763, 3111.94925847, ..., 3300.61782262,
        3297.43206611, 3300.28787972],
       [3110.87      , 3109.22924749, 3106.59995608, ..., 3113.96509913,
        3113.69332945, 3112.9845784 ]])

In [104]:
pp = (K - S[-1,:])>0
bpp = np.mean(np.maximum(pp.astype(int),0))
print('Binary put', str(bpp))

Binary put 0.5814418558144185


In [120]:
def euro_option_bsm(S, K, T, r, q, sig, payoff):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: risk free rate
    #q: continuous dividend yield
    #vol: volatility of underlying asset
    #payoff: call or put
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
    if payoff == "call":
        option_value = S * np.exp(-q * T) * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        option_value =  - S * np.exp(-q * T) * si.norm.cdf(-d1, 0.0, 1.0) + K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return option_value

In [122]:
delta(3110.87, 3100, 4/52, 0.0174, 0, 0.3575, 'put')


-0.4608162138768531

In [116]:
def gamma(S, K, T, r, q, sig, payoff):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sig ** 2) * T) / (sig * np.sqrt(T))
        gamma = np.exp(-r * T) * si.norm.pdf(d1, 0.0, 1.0) / (sig * S * np.sqrt(T))
    
    return gamma

In [123]:
gamma(3110.87, 3100, 4/52, 0.0174, 0, 0.3575, 'put')

0.0012854078883122724

In [124]:
def theta(S, K, T, r, q, vol, payoff):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    if payoff == "call":
        theta = vol * S * np.exp(-q * T) * si.norm.pdf(d1, 0.0, 1.0) / (2 * np.sqrt(T)) - q * S * np.exp(-q * T) * si.norm.cdf(d1, 0.0, 1.0) + r * K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        theta = vol * S * np.exp(-q * T) * si.norm.pdf(-d1, 0.0, 1.0) / (2 * np.sqrt(T)) - q * S * np.exp(-q * T) * si.norm.cdf(-d1, 0.0, 1.0) + r * K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return theta

In [125]:
theta(3110.87, 3100.00, 4/52, 0.0174, 0, 0.3575, 'put')

822.9414199007169

In [126]:
def rho(S, K, T, r, q, vol, payoff):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    if payoff == "call":
        rho =  K * T * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        rho = - K * T * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return rho

In [127]:
rho(3110.87, 3100, 4/52, 0.0174, 0.0, 0.3575, 'put')

-119.14492089318998

In [128]:
def vega(S, K, T, r, q, vol, payoff):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    vega = S * np.sqrt(T) * np.exp(-q * T) * si.norm.pdf(d1, 0.0, 1.0)
    
    return vega

In [129]:
vega(3110.87, 3100.00, 4/52, 0.0174, 0.0, 0.3575, 'put')

342.54581551142655

In [130]:
def speed(S, K, T, r, q, vol, payoff):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    speed = np.exp(-q * T) * si.norm.pdf(d1, 0.0, 1.0) / ((vol **2) * (S**2) * np.sqrt(T)) * (d1 + vol * np.sqrt(T))
    
    return speed

In [131]:
speed(3110.87, 3100.00, 4/52, 0.0174, 0.0, 0.3575, 'put')

2.286114961509202e-07